In [1]:
import os
import numpy as np
import datetime
import time
import pandas
import pdb
from scipy.interpolate import interp1d
import pickle
import statistics
from scipy.signal import butter, filtfilt
from scipy.stats import zscore, kurtosis, skew
from utils import  calculate_ellipse_area,create_ellipse, energy_peak, zero_crossing_rate, energy, sma, percentile

In [2]:
# file = "TJR/ID_303/36M_WALK_TEST/accelerometer.csv"
file = "TJR/App_Data.csv"
# walkBAC = pandas.read_csv(file, skiprows = 1)
walkBAC = pandas.read_csv(file)

In [3]:
sensorData = walkBAC.dropna()

In [4]:
sensorData

,ID,X,Y,Z,TimeStamp,Condition
0,303,-2.001553,9.696517,0.517148,13-02-2020 14:35:36.980,Healthy
1,303,-2.001553,9.696517,0.517148,13-02-2020 14:35:36.980,Healthy
2,303,-2.001553,9.696517,0.517148,13-02-2020 14:35:36.980,Healthy
3,303,-2.001553,9.696517,0.517148,13-02-2020 14:35:36.980,Healthy
4,303,-2.092532,9.749189,0.493206,13-02-2020 14:35:36.982,Healthy
...,...,...,...,...,...,...
157637,311,-2.240973,9.643845,0.469264,13-02-2020 15:12:26.291,LBP
157638,311,-2.240973,9.643845,0.469264,13-02-2020 15:12:26.291,LBP
157639,311,-2.188300,9.682152,0.536301,13-02-2020 15:12:26.299,LBP
157640,311,-2.188300,9.682152,0.536301,13-02-2020 15:12:26.299,LBP


In [5]:
finalData = sensorData.drop_duplicates()

In [6]:
date_time = finalData["TimeStamp"].str.split(" ", n = 1, expand = True)

In [7]:
date_time

,0,1
0,13-02-2020,14:35:36.980
4,13-02-2020,14:35:36.982
7,13-02-2020,14:35:36.984
11,13-02-2020,14:35:36.985
14,13-02-2020,14:35:36.987
...,...,...
157625,13-02-2020,15:12:26.258
157628,13-02-2020,15:12:26.270
157632,13-02-2020,15:12:26.278
157635,13-02-2020,15:12:26.291


In [8]:
finalData["Time"]= date_time[1] 

/Users/amishajindal28/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
finalData.drop(columns =["TimeStamp"], inplace = True)
# inplace does changes in the original dataset

/Users/amishajindal28/.pyenv/versions/3.7.4/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [10]:
finalData

,ID,X,Y,Z,Condition,Time
0,303,-2.001553,9.696517,0.517148,Healthy,14:35:36.980
4,303,-2.092532,9.749189,0.493206,Healthy,14:35:36.982
7,303,-2.173935,9.730036,0.579397,Healthy,14:35:36.984
11,303,-2.217031,9.739613,0.588974,Healthy,14:35:36.985
14,303,-2.193089,9.572019,0.574608,Healthy,14:35:36.987
...,...,...,...,...,...,...
157625,311,-2.470816,9.725247,0.359130,LBP,15:12:26.258
157628,311,-2.446874,9.753978,0.277727,LBP,15:12:26.270
157632,311,-2.427721,9.749189,0.430956,LBP,15:12:26.278
157635,311,-2.240973,9.643845,0.469264,LBP,15:12:26.291


In [11]:
# sensorData = walkBAC[['X', 'Y', 'Z']]

In [12]:
# sensorData

In [13]:
# semgementation_seconds = 8
# signaltype = '36M_WALK_TEST'

In [14]:
# seg_seconds = 8.00
# seg_seconds = 5.00
# seg_seconds = 10.00
seg_seconds = 2.00
# samples = 128

# samplingType = 'segments'
# overlap_rate = 0.5

def butter_lowpass_filter(data):
  T = 5.0         # Sample Period
  fs = 50.0       # sample rate, Hz
  cutoff = 2      # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
  nyq = 0.5 * fs  # Nyquist Frequency
  order = 3       # sin wave can be approx represented as quadratic
  n = int(T * fs) # total number of samples

  noise = 1.5*np.cos(9*2*np.pi*T) + 0.5*np.sin(12.0*2*np.pi*T)
  normal_cutoff = cutoff / nyq

  x = data['X'] + noise
  y = data['Y'] + noise
  z = data['Z'] + noise

  b, a = butter(order, normal_cutoff, btype='low', analog=False)
  data['X'] = filtfilt(b, a, x)
  data['Y'] = filtfilt(b, a, y)
  data['Z'] = filtfilt(b, a, z) 
  return data

In [15]:
data = butter_lowpass_filter(finalData)
print(data.head())

     ID         X          Y         Z Condition          Time
0   303 -0.453663  11.347826  2.110922   Healthy  14:35:36.980
4   303 -0.495761  11.270878  2.056700   Healthy  14:35:36.982
7   303 -0.539317  11.187680  1.998793   Healthy  14:35:36.984
11  303 -0.582127  11.104188  1.940741   Healthy  14:35:36.985
14  303 -0.622249  11.025850  1.885853   Healthy  14:35:36.987


/Users/amishajindal28/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/amishajindal28/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/amishajindal28/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [16]:
# X_train = data[data['ID'] < 309]
# y_train = X_train['Condition']
# X_train = X_train.drop(['Condition','TimeStamp'],axis=1)

# X_test = data[data['ID'] >= 309]
# y_test = X_test['Condition']
# X_test = X_test.drop(['Condition','TimeStamp'],axis=1)

In [17]:
# data.iloc[0]['Time']

In [18]:
# def segmenting(data):
#     start = 0
#     end = 0
#     i = 0
#     for s in data['Time']:
#         if data.iloc[i]['ID'] == data.iloc[i+1]['ID']:
# #             print(s)
# #             print(data.iloc[start]['Time'])
# #             diff = s - data.iloc[start]['Time']
#             r = data.iloc[start]['Time']
# #             print(r)
#             t1 = pandas.to_datetime(s)
# #             print(s)
#             t2 = pandas.to_datetime(r)
# #             print(r)
#             diff = pandas.Timedelta(t1 - t2).total_seconds()
# # #             diff = (s - data.iloc[start]['Time']).astype('timedelta64[h]')
#             print(diff)
#             i += 1
# #             if(diff == seg_seconds)
# #         print(i, s)

In [19]:
# segmenting(data)

In [20]:
def find_features(uid, x, y, z, c):
    if c == 'Healthy':
        problem = 0
    elif c == 'OA':
        problem = 1
    else:
        problem = 2
    acc_x_seg = x
    acc_y_seg = y
    acc_z_seg = z
    acc_x_avg = np.average(acc_x_seg)
    acc_y_avg = np.average(acc_y_seg)
    acc_z_avg = np.average(acc_z_seg)
    acc_rx_ct = max(acc_x_seg) - min(acc_x_seg)
    acc_ry_ct = max(acc_y_seg) - min(acc_y_seg)
    acc_rz_ct = max(acc_z_seg) - min(acc_z_seg)
    acc_stdx_ct   = statistics.stdev(acc_x_seg)
    acc_stdy_ct = statistics.stdev(acc_y_seg)
    acc_stdz_ct = statistics.stdev(acc_z_seg)
    acc_rmsx_ct = np.sqrt(np.mean(acc_x_seg ** 2))
    acc_rmsy_ct = np.sqrt(np.mean(acc_y_seg ** 2))
    acc_rmsz_ct = np.sqrt(np.mean(acc_z_seg ** 2))           
    acc_skx_ct = skew(acc_x_seg)
    acc_sky_ct = skew(acc_y_seg)
    acc_skz_ct = skew(acc_z_seg)
    acc_x_energy = energy(acc_x_seg)
    acc_y_energy = energy(acc_y_seg)
    acc_z_energy = energy(acc_z_seg)
    Walk = []
    Walk.append([uid,acc_x_avg,acc_y_avg,acc_z_avg,\
                 acc_rx_ct,acc_ry_ct,acc_rz_ct,\
                    acc_stdx_ct,acc_stdy_ct,acc_stdz_ct,\
                    acc_rmsx_ct,acc_rmsy_ct,acc_rmsz_ct,\
                    acc_skx_ct,acc_sky_ct,acc_skz_ct,\
                    acc_x_energy,acc_y_energy,acc_z_energy,\
                problem])
#     Walk.append([acc_rx_ct ,acc_ry_ct ,acc_rz_ct ,\
#                     acc_stdx_ct ,acc_stdy_ct ,acc_stdz_ct ,\
#                     acc_rmsx_ct ,acc_rmsy_ct ,acc_rmsz_ct ,\
#                     acc_kurtx_ct,acc_kurty_ct,acc_kurtz_ct,\
#                     acc_energypeakX_ct,acc_energypeakY_ct,acc_energypeakZ_ct,\
#                     acc_freqspeakX_ct,acc_freqspeakY_ct,acc_freqspeakZ_ct,\
#                     acc_skx_ct ,acc_sky_ct ,acc_skz_ct ,\
#                     acc_a1 ,acc_b1 ,acc_c1,\
#                     acc_x_energy ,acc_y_energy ,acc_z_energy,\
#                 problem])
    WholeWalk.append(Walk)

In [21]:
data.count()

ID           45148
X            45148
Y            45148
Z            45148
Condition    45148
Time         45148
dtype: int64

In [22]:
WholeWalk = []
def segmenting(data):
    start = 0
    end = 0
    i = 0
    for s in data['Time']:
        if i < 45147:
            if data.iloc[i]['ID'] == data.iloc[i+1]['ID']:
    #             print(s)
    #             print(data.iloc[start]['Time'])
    #             diff = s - data.iloc[start]['Time']
                r = data.iloc[start]['Time']
    #             print(r)
                t1 = pandas.to_datetime(s)
    #             print(s)
                t2 = pandas.to_datetime(r) 
    #             t3 = pandas.to_timedelta(r) + pandas.Timedelta(seconds=8)
    #             print(t3)
                diff = pandas.Timedelta(t1 - t2).total_seconds()
    #             diff = pandas.Timedelta(t1 - t2)
#                 print(diff)
    #             print(diff - seg_seconds)
    #             diff = t1 - t2
    #             print(pandas.to_timedelta(diff, unit='s'))
    #             if pandas.to_timedelta(diff, unit='s') < pandas.to_timedelta(t3, unit='s'):
    #             if t3 - diff > 0
                if diff - seg_seconds >= 0:
                    end = i
    #                 print(end)
                    uid = data.iloc[i]['ID']
                    x = data["X"][start:end]
                    y = data["Y"][start:end]
                    z = data["Z"][start:end]
                    c = data.iloc[i]["Condition"]
                    find_features(uid, x, y, z, c)
                    start = end + 1
                    #ignores the segment which is less than 8 seconds
    # #             diff = (s - data.iloc[start]['Time']).astype('timedelta64[h]')
    #             print(diff)
                i += 1
            else:
                i += 1
        else:
            break
#     return segments
#             if(diff == seg_seconds)
#         print(i, s)

In [23]:
# segmenting(data)

In [24]:
# from scipy import stats


# def segment_signal(data,window_size = 10):
#     segments = np.empty((0,window_size,3))
#     labels = np.empty((0))
#     for (start, end) in windows(data["TimeStamp"], window_size):
        
#         if(len(data["TimeStamp"][start:end]) == window_size):
#             segments = np.vstack([segments,np.dstack([x,y,z])])
#             labels = np.append(labels,stats.mode(data["Condition"][start:end])[0][0])
#     return segments, labels

In [25]:
# (df.fr-df.to).astype('timedelta64[h]')

In [26]:
segmenting(data)

In [27]:
WholeWalk

[[[303,
   -0.4015396526378184,
   11.144810619772477,
   2.027110957615553,
   7.370881115402636,
   7.679001825250483,
   0.927328052782457,
   1.3664454106487371,
   1.4162766110253326,
   0.23643389649042126,
   1.42097280830141,
   11.233998099132748,
   2.0407849186852727,
   0.9650800577913906,
   0.715961858678943,
   0.023103312155929555,
   407.8710718302633,
   25492.94808484627,
   841.290223035317,
   0]],
 [[303,
   -0.006305445414703297,
   11.249858449765789,
   2.0836207626044754,
   9.687715291776922,
   10.184351803072868,
   2.0465919816055598,
   2.2807174204415053,
   2.5088218550756896,
   0.4171957277314856,
   2.274959480976104,
   11.524830298817081,
   2.124770271276103,
   -0.23718344773846278,
   0.02093796704928552,
   -0.05891040383311648,
   1024.7372467364464,
   26298.699256473377,
   893.9004437283468,
   0]],
 [[303,
   -0.0758409898846557,
   11.080079196765569,
   2.072468739167432,
   9.009249327777523,
   12.886876523822263,
   1.2417753627240375

In [28]:
z = len(WholeWalk)

In [29]:
z

228

In [30]:
matrix = np.array(WholeWalk).reshape(z, 20) 

In [31]:
matrix

array([[ 3.03000000e+02, -4.01539653e-01,  1.11448106e+01, ...,
         2.54929481e+04,  8.41290223e+02,  0.00000000e+00],
       [ 3.03000000e+02, -6.30544541e-03,  1.12498584e+01, ...,
         2.62986993e+04,  8.93900444e+02,  0.00000000e+00],
       [ 3.03000000e+02, -7.58409899e-02,  1.10800792e+01, ...,
         2.63932787e+04,  8.82578869e+02,  0.00000000e+00],
       ...,
       [ 3.11000000e+02, -1.38031592e+00,  1.09319697e+01, ...,
         2.38548047e+04,  1.50837690e+03,  2.00000000e+00],
       [ 3.11000000e+02, -1.00180785e+00,  1.09152879e+01, ...,
         2.37590985e+04,  2.01252169e+03,  2.00000000e+00],
       [ 3.11000000e+02, -1.31459456e+00,  1.08776496e+01, ...,
         2.38336740e+04,  1.65946131e+03,  2.00000000e+00]])

In [32]:
model_data = pandas.DataFrame(matrix, columns=['ID','X_Avg','Y_Avg','Z_Avg','X_Range','Y_Range','Z_Range','X_SD','Y_SD','Z_SD','X_RMS','Y_RMS','Z_RMS','X_Skew','Y_Skew','Z_Skew','X_Energy','Y_Energy','Z_Energy','Class'])

In [33]:
X_train = model_data[model_data['ID'] < 309]
y_train = X_train['Class']
X_train = X_train.drop(['Class'],axis=1)

X_test = model_data[model_data['ID'] >= 309]
y_test = X_test['Class']
X_test = X_test.drop(['Class'],axis=1)

In [34]:
# y_train = X_train['Class']
# X_train = X_train.drop(['Class'],axis=1)

In [35]:
# WholeWalk

In [36]:
# WholeWalk = []
# segmenting(X_test)
# X_test = DataFrame(WholeWalk, columns=['X_Avg','Y_Avg','Z_Avg','X_Range','Y_Range','Z_Range','X_SD','Y_SD','Z_SD','X_RMS','Y_RMS','Z_RMS','X_Skew','Y_Skew','Z_Skew','X_Energy','Y_Energy','Z_Energy','Class'])

In [37]:
# y_test = X_train['Class']
# X_test = X_train.drop(['Class'],axis=1)

In [38]:
# WholeWalk

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix, accuracy_score

In [40]:
parameters = {'bootstrap': True,
              'min_samples_leaf': 3,
              'n_estimators': 50, 
              'min_samples_split': 10,
              'max_features': 'sqrt',
              'max_depth': 6,
              'max_leaf_nodes': None}

In [41]:
RF_model = RandomForestClassifier(**parameters)

In [42]:
RF_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=6, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [43]:
y_pred = RF_model.predict(X_test)
score = accuracy_score(y_test ,y_pred)
print(score)

0.5


In [44]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      0.50      0.67        16
         1.0       0.00      0.00      0.00        30
         2.0       0.49      1.00      0.66        30

    accuracy                           0.50        76
   macro avg       0.50      0.50      0.44        76
weighted avg       0.40      0.50      0.40        76



In [45]:
from sklearn.svm import SVC

In [46]:
clf = SVC(gamma='auto')
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [47]:
y_pred = clf.predict(X_test)
score = accuracy_score(y_test ,y_pred)
print(score)

0.39473684210526316


In [48]:
# os.chdir('/Users/amishajindal28/Desktop/DR/TJR')
# print('saving into'+os.getcwd())
# with open('Data_features_seg.csv', 'wb') as f:
#     f.write(WholeWalk)

In [49]:
# np.savetxt('data.csv', WholeWalk[0], delimiter=',')

In [50]:
# df.to_csv(r'C:\Users\Ron\Desktop\export_dataframe.csv', index = False, header=True)
# WholeWalk.to_csv(r'\TJR\data.csv', index = False)

In [51]:
# import csv

# with open('Data_features_seg.csv', 'w', newline='') as myfile:
#      wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#      wr.writerow(WholeWalk)

In [52]:
# dataDir = "/TJR"
# for subject in next(os.walk(dataDir))[1]:
#     subjectBAC = []
#     print("Subject : " + subject)

In [53]:
# for subject in next(os.walk(dataDir)):
#     print("Subject : " + subject)